In [1]:
%reload_ext autoreload
%autoreload 2

In [28]:
import chess
from playing.utils import board_to_piece_uci_moves
from playing.agents import GPTEloAgent, NegaMaxMaterialGPTEloAgent
from lightning_training import LightningGPT

from data_process import tokenizers
from nanoGPT.model import GPTConfig


In [3]:
material_pair_tokenizer = tokenizers.FullMoveEloMaterialPairTokenizer()

material_tokenizer = tokenizers.FullMoveEloMaterialTokenizer()

base_tokenizer = tokenizers.FullMoveTokenizerWithElo()

material_pair_model_config = GPTConfig(
    block_size=604,
    vocab_size=len(material_pair_tokenizer.vocab),
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

material_model_config = GPTConfig(
    block_size=604,
    vocab_size=material_tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

base_model_config = GPTConfig(
    block_size=302,
    vocab_size=base_tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

test_start_token = 21
test_token_step = 2

test_start_token_base = test_start_token // 2 + 1
test_token_step_base = 1

material_checkpoint = "./models/full_training/elo_material_ignore_material_prediction/epoch=9-step=1250000.ckpt"
# material_single_checkpoint = "./models/small_training/elo_material_ignore_material_prediction/epoch=4-step=78125.ckpt"


material_model = LightningGPT.load_from_checkpoint(
    material_checkpoint,
    config=material_model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
    trainig_ignore_first_n_targets=test_start_token,
    training_target_step=2,
    tokenizer=material_tokenizer,
    masked_elo_test=True
)

material_pair_checkpoint = "./models/full_training/elo_material_pair_ignore_material_prediction/epoch=9-step=1250000.ckpt"

material_pair_model = LightningGPT.load_from_checkpoint(
    material_pair_checkpoint,
    config=material_pair_model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
    trainig_ignore_first_n_targets=test_start_token,
    training_target_step=2,
    tokenizer=material_pair_tokenizer,
    masked_elo_test=True
)

base_checkpoint = "./models/full_training/masked_elo/epoch=9-step=1250000.ckpt"

base_model = LightningGPT.load_from_checkpoint(
    base_checkpoint,
    config=base_model_config,
    test_start_token=test_start_token_base,
    test_token_step=test_token_step_base,
    trainig_ignore_first_n_targets=test_start_token_base,
    training_target_step=test_token_step_base,
    tokenizer=base_tokenizer,
    masked_elo_test=True
)

base_no_mask_checkpoint = "./models/full_training/adaptive_elo/epoch=9-step=1250000.ckpt"

base_no_mask_model = LightningGPT.load_from_checkpoint(
    base_no_mask_checkpoint,
    config=base_model_config,
    test_start_token=test_start_token_base,
    test_token_step=test_token_step_base,
    trainig_ignore_first_n_targets=test_start_token_base,
    training_target_step=test_token_step_base,
    tokenizer=base_tokenizer,
    masked_elo_test=True
)

number of parameters: 27.34M
number of parameters: 28.15M
number of parameters: 27.29M
number of parameters: 27.29M


In [7]:
model = material_model.model
agent = GPTEloAgent(model=model, model_elo=1000, use_material_tokens=True)

board = chess.Board()
agent.play(board)

Move.from_uci('e2e4')

In [25]:
import pandas as pd
from data_process.utils import add_elo_token_to_games, remove_last_player_material_token


headers = ["index", "id", "date", "white_elo", "black_elo", "result", "ply", "ply_30s", "piece_uci"]

games_df = pd.read_csv("./data/test.csv", delimiter=";", header=None, names=headers)

games_df = games_df.sample(frac=0.1, random_state=42)

games = remove_last_player_material_token(games_df.piece_uci)
games = add_elo_token_to_games(games, games_df.white_elo, games_df.black_elo)

games_df

,index,id,date,white_elo,black_elo,result,ply,ply_30s,piece_uci
75721,99034768,j8dut9JL,2025.01.31,2425,2427,1-0,85,68,Pd2d4 39 39 Ng8f6 39 39 Ng1f3 39 39 Pc7c5 39 3...
80184,7380677,UMIvcBzB,2024.12.03,1919,2020,1-0,67,47,Pd2d4 39 39 Ng8f6 39 39 Nb1c3 39 39 Pc7c5 39 3...
19864,14462158,RP3reGqr,2024.12.05,2135,2144,1-0,41,-1,Ng1f3 39 39 Pc7c6 39 39 Pg2g3 39 39 Pd7d5 39 3...
76699,99355390,OC9pukWM,2025.01.31,2402,2433,1-0,53,-1,Pc2c4 39 39 Pe7e6 39 39 Nb1c3 39 39 Pd7d5 39 3...
92991,14089339,BL4die9s,2024.12.05,1200,1143,1-0,45,-1,Pe2e4 39 39 Pd7d5 39 39 Pe4d5 39 38 Qd8d5 38 3...
...,...,...,...,...,...,...,...,...,...
5002,97062,?,2021.12.07,2726,2574,1-0,9,-1,Pd2d4 39 39 Pd7d5 39 39 Nb1c3 39 39 Ng8f6 39 3...
30151,45151922,dUvdD4GK,2024.12.15,2258,2163,0-1,80,-1,Pc2c4 39 39 Pe7e6 39 39 Ng1f3 39 39 Pd7d5 39 3...
93194,14092632,7yOJ8xPr,2024.12.05,1051,1129,1-0,45,-1,Pe2e4 39 39 Pe7e5 39 39 Pd2d3 39 39 Nb8c6 39 3...
73199,98330191,Zw7vLYf0,2025.01.31,2418,2414,0-1,48,22,Pe2e4 39 39 Pd7d5 39 39 Pe4d5 39 38 Qd8d5 38 3...


In [30]:
def elo_agent_prediction(agent, piece_uci, white_elo, black_elo):
    if isinstance(piece_uci, str):
        piece_uci = piece_uci.split(" ")
    if isinstance(white_elo, int):
        white_elo = str(white_elo)
    if isinstance(black_elo, int):
        black_elo = str(black_elo)

    board = chess.Board()

    correct_predictions = 0
    all_predictions = 0

    for move in piece_uci:
        if board.turn == chess.WHITE:
            agent.model_elo = white_elo
        else:
            agent.model_elo = black_elo

        predicted_move = agent.play(board)
        real_move = chess.Move.from_uci(move[1:])
        if predicted_move == real_move:
            correct_predictions += 1
        all_predictions += 1
        board.push(chess.Move.from_uci(move[1:]))

    return correct_predictions, all_predictions

In [33]:
from tqdm import tqdm

def test_elo_agent(agent, games_df):
    correct_predictions = 0
    all_predictions = 0

    for index, row in tqdm(list(games_df.iterrows())):
        piece_uci = row["piece_uci"].split(" ")[::3]
        white_elo = int(row["white_elo"]) // 100 * 100
        black_elo = int(row["black_elo"]) // 100 * 100

        correct, all_moves = elo_agent_prediction(agent, piece_uci, white_elo, black_elo)
        correct_predictions += correct
        all_predictions += all_moves

    return correct_predictions, all_predictions

In [ ]:
games_df_sample = games_df.sample(n=10, random_state=42)
agent = GPTEloAgent(model=model, model_elo=1500, use_material_tokens=True)

corect_prediction, all_predictions = test_elo_agent(agent, games_df_sample)

print(f"{corect_prediction} / {all_predictions} = {corect_prediction / all_predictions:.2%}")



100%|██████████| 10/10 [00:17<00:00,  1.77s/it]

376 / 714 = 52.66%


In [73]:
search_agent = NegaMaxMaterialGPTEloAgent(model=model, depth=1, model_elo=1500, use_material_tokens=True)

In [74]:
board = chess.Board()
search_agent.play(board)

Move.from_uci('e2e4')

In [75]:
corect_prediction, all_predictions = test_elo_agent(search_agent, games_df_sample)

print(f"{corect_prediction} / {all_predictions} = {corect_prediction / all_predictions:.2%}")

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]

329 / 714 = 46.08%
